In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)


# Query (by Permit Sub Types)

### SQL Code Used for Query:

```WITH demo AS (
    SELECT 
        permittype, permitsubtype, apn as apn1, location, purpose, dateissued, value, status 
	FROM permit
	WHERE permitsubtype like 'BLAST%' or permitsubtype like 'DEMO%'
),
improvements AS (
    SELECT permittype as permittype_imp, permitsubtype as permitsubtype_imp, apn as apn2, purpose as purpose_imp, dateissued as dateissued_imp, value as value_imp, status as status_imp
	FROM permit
	WHERE permitsubtype like 'ACCESSORY APARTMENT' 
	OR permitsubtype LIKE 'ACCESSORY STRUCTURE, CARPORT' 
	OR permitsubtype LIKE 'ACCESSORY STRUCTURE, DECKS' 
	OR permitsubtype LIKE 'ACCESSORY STRUCTURE, GARAGE' 
	OR permitsubtype LIKE 'ACCESSORY STRUCTURE, POOLS'
	OR permitsubtype LIKE 'ACCESSORY STRUCTURE, POOLS - RESIDENTIAL' 
	OR permitsubtype LIKE 'ACCESSORY STRUCTURE, SHED / STORAGE BLDG' 
	OR permitsubtype LIKE 'ALARM PERMIT RESIDENTIAL' 
	OR permitsubtype LIKE 'DETACHED ACCESSORY DWELLING UNIT'
)
SELECT 
	permittype, 
	permitsubtype, 
	apn as apn1, 
	location, 
	purpose, 
	dateissued, 
	value, 
	status, 
	permittype_imp,
	permitsubtype_imp,
	purpose_imp,
	dateissued_imp,
	value_imp, 
	status_imp, 
	featuretype,
	owndate,
	saleprice, 
	propaddr, 
	ownzip, 
	propdate,
	ludesc, 
	landappr, 
	imprappr, 
	totlappr, 
	landassd, 
	imprassd,
	totlassd,
	assessdate  
FROM demo
INNER JOIN improvements ON demo.apn1 = improvements.apn2
INNER JOIN property_updated ON demo.apn1 = property_updated.apn```

In [2]:
#read in data
Q2 = pd.read_csv('Q2 full data.csv')

In [3]:
Q2

,permittype,permitsubtype,apn1,location,purpose,dateissued,value,status,permittype_imp,permitsubtype_imp,purpose_imp,dateissued_imp,value_imp,status_imp,featuretype,owndate,saleprice,propaddr,ownzip,propdate,ludesc,landappr,imprappr,totlappr,landassd,imprassd,totlassd,assessdate
0,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,08208022000,414 HART AVE 37206,WORK ON THIS DEMOLITION PERMIT MUST COMMENCE W...,2015-01-15,4000.0,DONE,ALARM PERMIT,ALARM PERMIT RESIDENTIAL,NaN,NaN,NaN,DONE,LOT,2015-12-18,399000.0,414 HART AVE,37206,1970-08-28,SINGLE FAMILY,145000,369600,514600,36250,92400,128650,2021-01-01
1,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,13101004500,4225 LINDAWOOD DR 37215,WORK ON THIS DEMOLITION PERMIT MUST COMMENCE W...,2011-03-04,7500.0,EXPIRED,ALARM PERMIT,ALARM PERMIT RESIDENTIAL,NaN,NaN,NaN,DONE,LOT,2012-02-23,1390000.0,4225 LINDAWOOD DR,37215,1972-05-26,SINGLE FAMILY,562500,1766800,2329300,140625,441700,582325,2021-01-01
2,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,13101004500,4225 LINDAWOOD DR 37215,WORK ON THIS DEMOLITION PERMIT MUST COMMENCE W...,2011-03-04,7500.0,EXPIRED,ALARM PERMIT,ALARM PERMIT RESIDENTIAL,NaN,NaN,NaN,DONE,LOT,2012-02-23,1390000.0,4225 LINDAWOOD DR,37215,1972-05-26,SINGLE FAMILY,562500,1766800,2329300,140625,441700,582325,2021-01-01
3,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,13104022000,1144 BROOKMEADE DR 37204,TO DEMOLISH EXISTING RESIDENCE AND THREE (3) D...,2020-02-18,15000.0,EXPIRED,BUILDING USE & OCCUPANCY,"ACCESSORY STRUCTURE, POOLS - RESIDENTIAL",TO CONSTRUCT A 14FT X 38FT SWIMMING POOL WITH ...,2020-06-25,NaN,ISSUED,LOT,2020-01-28,NaN,1144 BROOKMEADE DR,37027,1967-07-19,SINGLE FAMILY,333500,1645500,1979000,83375,411375,494750,2021-01-01
4,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,08203019600,1107 STOCKELL ST 37207,WORK ON THIS DEMOLITION PERMIT MUST COMMENCE W...,2020-04-29,6000.0,EXPIRED,BUILDING RESIDENTIAL - NEW,DETACHED ACCESSORY DWELLING UNIT,RESTRICTIVE COVENANT RECORDED 20210122-0009740...,2021-03-08,NaN,ISSUED,LOT,2020-02-14,182500.0,1107 STOCKELL ST,37208,1954-06-29,VACANT RESIDENTIAL LAND,150000,0,150000,37500,0,37500,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,09107040500,4805B MICHIGAN AVE 37209,TO DEMOLISH EXISTING SINGLE FAMILY RESIDENCE. ...,2015-02-25,5000.0,EXPIRED,BUILDING RESIDENTIAL - NEW,"ACCESSORY STRUCTURE, GARAGE",RESTRICTIVE COVENANT FOR DETACHED STRUCTURE ON...,2015-02-25,13526.67,DONE,LOT,2019-12-19,535000.0,4805 A MICHIGAN AVE,37209,2015-02-21,SINGLE FAMILY,166500,392800,559300,41625,98200,139825,2021-01-01
3758,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,10401031100,3605 NEBRASKA AVE 37209,** TO DEMOLISH EXISTING SGL. FAMILY RESIDENT...,2015-01-07,4500.0,EXPIRED,BUILDING RESIDENTIAL - NEW,"ACCESSORY STRUCTURE, GARAGE",RESTRICTIVE COVENANT FOR DETACHED STRUCTURE ON...,2015-01-29,19784.16,EXPIRED,LOT,2020-12-16,NaN,3605 NEBRASKA AVE,37209,2015-01-23,SINGLE FAMILY,290000,595400,885400,72500,148850,221350,2021-01-01
3759,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,11000014300,3587 EARHART RD 37122,WORK ON THIS DEMOLITION PERMIT MUST COMMENCE W...,2012-12-11,5000.0,EXPIRED,BUILDING RESIDENTIAL - NEW,"ACCESSORY STRUCTURE, GARAGE",TO CONSTRUCT A NEW DETACHED 30 X 60 WITH A HEI...,2013-01-28,103734.00,EXPIRED,ACREAGE TRACT,2012-12-21,0.0,3583 EARHART RD,37122,2012-12-21,SINGLE FAMILY,197000,552500,749500,49250,138125,187375,2021-01-01
3760,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,09402003600,1619 ELECTRIC AVE 37206,DEMOLITION OF EXISTING HOUSE AND ALL OUTBUILDI...,2017-04-13,7000.0,EXPIRED,ALARM PERMIT,ALARM PERMIT RESIDENTIAL,NaN,NaN,NaN,DONE,LOT,2014-01-02,0.0,1619 ELECTRIC AVE,37206,1970-01-19,SINGLE FAMILY,180000,776400,956400,45000,194100,239100,2021-01-01


In [4]:
# Modify date columns to date time types
Q2[['dateissued', 'dateissued_imp']] = Q2[['dateissued', 'dateissued_imp']].apply(pd.to_datetime)


# Want to create column to show duration from permit issued for demolition to permit issued for improvement
Q2['Duration'] = (Q2['dateissued_imp'] - Q2['dateissued']).dt.days


In [5]:
#filter data based on 2 years or less (730 days) from demolition permit to improvement permit
# This is to ensure we capture only permit that have demolition that was done in order to improve property.
#Q2_2_years = Q2[Q2['Duration'] <= 730 ]
Q2_2_years = Q2[(Q2.Duration <= 730) & (Q2.Duration >= 0) & (Q2.permittype_imp =="BUILDING RESIDENTIAL - NEW")]

In [6]:
Q2_2_years["location_zip"] = Q2_2_years["location"].str[-5:]

<ipython-input-6-3864d2121b58>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q2_2_years["location_zip"] = Q2_2_years["location"].str[-5:]


In [7]:
# sort by location
Q2_2_years.sort_values("location")

,permittype,permitsubtype,apn1,location,purpose,dateissued,value,status,permittype_imp,permitsubtype_imp,purpose_imp,dateissued_imp,value_imp,status_imp,featuretype,owndate,saleprice,propaddr,ownzip,propdate,ludesc,landappr,imprappr,totlappr,landassd,imprassd,totlassd,assessdate,Duration,location_zip
70,BUILDING BLASTING PERMIT,BLASTING PERMIT - 6 MONTHS,12100026900,100 HERON POINTE DR 37214,THIS PERMIT IS VALID FOR 6 MONTHS FROM THE DAT...,2014-01-02,0.0,EXPIRED,BUILDING RESIDENTIAL - NEW,"ACCESSORY STRUCTURE, GARAGE",TO CONSTRUCT PARKING GARAGES FOR APARTMENT BUI...,2014-10-15,301221.55,DONE,ACREAGE TRACT,2013-09-18,2539036.0,100 HERON POINTE DR,27401,2013-05-16,APARTMENT: LOW RISE (BUILT SINCE 1960),2280000,59879100,62159100,912000,23951640,24863640,2021-01-01,286.0,37214
822,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,08204013600,1001 N 6TH ST 37207,DEMOLISH 1080 SQ. FT. RESIDENTIAL STRUCTURE. W...,2019-12-16,3000.0,EXPIRED,BUILDING RESIDENTIAL - NEW,DETACHED ACCESSORY DWELLING UNIT,"CONSTRUCT SINGLE STORY DADU, 747 SQ. FT., WITH...",2019-12-30,NaN,ISSUED,LOT,2019-10-30,170000.0,1001 N 6TH ST,37207,1972-05-09,SINGLE FAMILY,172500,644200,816700,43125,161050,204175,2021-01-01,14.0,37207
997,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,11801018700,1002 GILMORE AVE 37204,WORK ON THIS DEMOLITION PERMIT MUST COMMENCE W...,2012-05-24,4000.0,EXPIRED,BUILDING RESIDENTIAL - NEW,"ACCESSORY STRUCTURE, GARAGE",CONSTRUCT A NEW DETACHED GARAGE AT 24 X 24..NO...,2012-06-27,21582.72,EXPIRED,LOT,2012-12-17,693494.0,1002 GILMORE AVE,37204.0,1974-04-15,SINGLE FAMILY,260000,743500,1003500,65000,185875,250875,2021-01-01,34.0,37204
379,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,10509044500,1003 BATE AVE 37204,WORK ON THIS DEMOLITION PERMIT MUST COMMENCE W...,2016-04-20,5000.0,ISSUED,BUILDING RESIDENTIAL - NEW,"ACCESSORY STRUCTURE, GARAGE",MHZC REVIEW/APPROVAL PRIOR TO PERMIT ISSUANCE....,2016-12-19,NaN,EXPIRED,LOT,2018-06-11,900000.0,1003 BATE AVE,37204,1969-03-11,SINGLE FAMILY,260000,768700,1028700,65000,192175,257175,2021-01-01,243.0,37204
2508,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,11801019700,1003 GILMORE AVE 37204,WORK ON THIS DEMOLITION PERMIT MUST COMMENCE W...,2011-09-09,6500.0,EXPIRED,BUILDING RESIDENTIAL - NEW,"ACCESSORY STRUCTURE, GARAGE",DETACHED 2 CAR GARAGE WITH OFFICE SPACE AND HA...,2011-12-09,28102.50,EXPIRED,LOT,2021-01-14,NaN,1003 GILMORE AVE,37027.0,1941-06-25,SINGLE FAMILY,260000,821200,1081200,65000,205300,270300,2021-01-01,91.0,37204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,11809018400,938 WOODMONT BLVD 37204,TO DEMOLISH EXISTING DETACHED GARAGE. NOT TO ...,2018-01-24,2000.0,EXPIRED,BUILDING RESIDENTIAL - NEW,"ACCESSORY STRUCTURE, GARAGE",TO CONSTRUCT 384SF DETACHED GARAGE TO THE REAR...,2018-01-24,NaN,EXPIRED,ACREAGE TRACT,2016-10-31,412000.0,938 WOODMONT BLVD,37204.0,1952-02-25,SINGLE FAMILY,280000,162900,442900,70000,40725,110725,2021-01-01,0.0,37204
3143,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,11813011900,945 CALDWELL LN 37204,DEMOLITION OF EXISTING SINGLE FAMILY HOUSE. N...,2020-06-30,7500.0,EXPIRED,BUILDING RESIDENTIAL - NEW,"ACCESSORY STRUCTURE, SHED / STORAGE BLDG",RESTRICTIVE CONVENANT FOR DETACHED STRUCTURE 2...,2020-08-14,NaN,ISSUED,LOT,2020-06-26,280000.0,945 CALDWELL LN,37027,1972-11-15,VACANT RESIDENTIAL LAND,273000,376700,649700,68250,94175,162425,2021-01-01,45.0,37204
41,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,08208029600,948 SEYMOUR AVE 37206,WORK ON THIS DEMOLITION PERMIT MUST COMMENCE W...,2012-11-15,6800.0,EXPIRED,BUILDING RESIDENTIAL - NEW,"ACCESSORY STRUCTURE, GARAGE",TO CONSTRUCT A 22 X 24 DETACHED GARAGE TO REAR...,2014-08-26,19784.16,DONE,LOT,2013-06-10,324900.0,948 SEYMOUR AVE,37206,1964-06-18,SINGLE FAMILY,125000,513300,638300,31250,128325,159575,2021

In [8]:
# Extract Year from Date/Time datatype
Q2_2_years['year'] = Q2_2_years['dateissued_imp'].dt.year

<ipython-input-8-58af87f64a7b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q2_2_years['year'] = Q2_2_years['dateissued_imp'].dt.year


In [9]:
# Aggreagate by zipcode and year
Q2_2_years_agg = Q2_2_years[['location_zip', 'year','permittype']].groupby(['location_zip', 'year']).agg(['count']).reset_index()

In [10]:
# slice out non-zipcode data
Q2_2_years_agg = Q2_2_years_agg[Q2_2_years_agg['location_zip'].str.contains(r'^37\d{3}$')]

In [11]:
Q2_2_years_agg

,location_zip,year,permittype
,,,count
0,37013,2007,24
1,37013,2015,8
2,37013,2019,1
3,37013,2020,1
4,37076,2009,1
5,37115,2012,1
6,37115,2016,2
7,37115,2017,1
8,37115,2019,1


In [12]:
#Filter zipcodes for year 2015
Q2_2_years_agg[Q2_2_years_agg['year']==2015].sort_values(('permittype', 'count'),ascending=False)

,location_zip,year,permittype
,,,count
82,37209,2015,36
51,37206,2015,18
26,37204,2015,9
59,37207,2015,9
1,37013,2015,8
38,37205,2015,8
103,37212,2015,5
68,37208,2015,4
119,37215,2015,3


In [13]:
#Filter zipcodes for year 2017
Q2_2_years_agg[Q2_2_years_agg['year']==2017].sort_values(('permittype', 'count'),ascending=False)

,location_zip,year,permittype
,,,count
84,37209,2017,37
53,37206,2017,10
40,37205,2017,7
105,37212,2017,5
28,37204,2017,4
61,37207,2017,2
70,37208,2017,2
120,37215,2017,2
7,37115,2017,1


In [14]:
#Filter zipcodes for year 2020
Q2_2_years_agg[Q2_2_years_agg['year']==2020].sort_values(('permittype', 'count'),ascending=False)

,location_zip,year,permittype
,,,count
87,37209,2020,24
108,37212,2020,7
31,37204,2020,6
56,37206,2020,4
43,37205,2020,3
9,37115,2020,2
64,37207,2020,2
90,37210,2020,2
3,37013,2020,1


In [15]:
#Filter zipcodes for year 2021
Q2_2_years_agg[Q2_2_years_agg['year']==2021].sort_values(('permittype', 'count'),ascending=False)

,location_zip,year,permittype
,,,count
88,37209,2021,5
57,37206,2021,4
65,37207,2021,3
44,37205,2021,2
74,37208,2021,2
32,37204,2021,1


# Additional Query (by Permit Types)

### SQL Code Used for Query:

```WITH demo AS (
    SELECT 
        permittype, permitsubtype, apn as apn1, location, purpose, dateissued, value, status 
	FROM permit
	WHERE permittype like 'BUILDING DEMOLITION PERMIT' and permitsubtype like 'DEMOLITION PERMIT - RESIDENTIAL' or permitsubtype like 'SINGLE FAMILY RESIDENCE'
),
improvements AS (
    SELECT permittype as permittype_imp, permitsubtype as permitsubtype_imp, apn as apn2, purpose as purpose_imp, dateissued as dateissued_imp, value as value_imp, status as status_imp
	FROM permit
	WHERE permittype like 'BUILDING RESIDENTIAL - ADDITION' 
	OR permittype LIKE 'BUILDING RESIDENTIAL - NEW' 
	OR permittype LIKE 'BUILDING RESIDENTIAL - REHAB' 
	OR permittype LIKE 'BUILDING RESIDENTIAL - ROOFING / SIDING' 
	OR permittype LIKE 'BUILDING RESIDENTIAL - SHELL'
	OR permittype LIKE 'BUILDING RESIDENTIAL - FOUNDATION' 
)
SELECT 
	permittype, 
	permitsubtype, 
	apn as apn1, 
	location, 
	purpose, 
	dateissued, 
	value, 
	status, 
	permittype_imp,
	permitsubtype_imp,
	purpose_imp,
	dateissued_imp,
	value_imp, 
	status_imp, 
	featuretype,
	owndate,
	saleprice, 
	propaddr, 
	ownzip, 
	propdate,
	ludesc, 
	landappr, 
	imprappr, 
	totlappr, 
	landassd, 
	imprassd,
	totlassd,
	assessdate  
FROM demo
INNER JOIN improvements ON demo.apn1 = improvements.apn2
INNER JOIN property_updated ON demo.apn1 = property_updated.apn```

In [16]:
#read in data
Q2_v2 = pd.read_csv('Q2 full data 2.csv')

C:\Users\alvin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
Q2_v2

,permittype,permitsubtype,apn1,location,purpose,dateissued,value,status,permittype_imp,permitsubtype_imp,purpose_imp,dateissued_imp,value_imp,status_imp,featuretype,owndate,saleprice,propaddr,ownzip,propdate,ludesc,landappr,imprappr,totlappr,landassd,imprassd,totlassd,assessdate
0,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,00200004500,131 EAST END RD 37072,CONSTRUCTION OF NEW SINGLE FAMILY RESIDENCE. ...,NaN,NaN,EXPIRED,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,CONSTRUCTION OF NEW SINGLE FAMILY RESIDENCE. ...,NaN,NaN,EXPIRED,ACREAGE TRACT,2019-12-09,272500.0,131 EAST END RD,37072,1986-09-19,VACANT RURAL LAND,24300.0,261700.0,286000.0,6075.0,65425.0,71500.0,2021-01-01
1,BUILDING RESIDENTIAL - ROOFING / SIDING,SINGLE FAMILY RESIDENCE,00300001500,00300001500 TO INSTALL GUTTERS AND SOFFIT TO T...,ACTIVITY-TYPE: A01 PURPOSE: TO INSTALL GUTTERS...,1999-04-20,3200.0,DONE,BUILDING RESIDENTIAL - NEW,NaN,ACTIVITY-TYPE: A02 PURPOSE: TO CONSTRUCT A 24'...,1988-04-11,15948.0,DONE,ACREAGE TRACT,2006-12-12,0.0,2454 TINNIN RD,37072,1986-11-06,SINGLE FAMILY,51200.0,206200.0,257400.0,12800.0,51550.0,64350.0,2021-01-01
2,BUILDING RESIDENTIAL - ROOFING / SIDING,SINGLE FAMILY RESIDENCE,00300001500,00300001500 TO INSTALL GUTTERS AND SOFFIT TO T...,ACTIVITY-TYPE: A01 PURPOSE: TO INSTALL GUTTERS...,1999-04-20,3200.0,DONE,BUILDING RESIDENTIAL - ROOFING / SIDING,SINGLE FAMILY RESIDENCE,ACTIVITY-TYPE: A01 PURPOSE: TO INSTALL GUTTERS...,1999-04-20,3200.0,DONE,ACREAGE TRACT,2006-12-12,0.0,2454 TINNIN RD,37072,1986-11-06,SINGLE FAMILY,51200.0,206200.0,257400.0,12800.0,51550.0,64350.0,2021-01-01
3,BUILDING RESIDENTIAL - ROOFING / SIDING,SINGLE FAMILY RESIDENCE,00300001500,00300001500 TO INSTALL GUTTERS AND SOFFIT TO T...,ACTIVITY-TYPE: A01 PURPOSE: TO INSTALL GUTTERS...,1999-04-20,3200.0,DONE,BUILDING RESIDENTIAL - NEW,NaN,PURPOSE: TO CONSTRUCT A 24' X 40' DETACHED GAR...,NaN,17280.0,DONE,ACREAGE TRACT,2006-12-12,0.0,2454 TINNIN RD,37072,1986-11-06,SINGLE FAMILY,51200.0,206200.0,257400.0,12800.0,51550.0,64350.0,2021-01-01
4,BUILDING RESIDENTIAL - REHAB,SINGLE FAMILY RESIDENCE,00300003900,"00300003900 TO MAKE GENERAL REPAIRS TO EX RES,...",ACTIVITY-TYPE: A01 PURPOSE: TO MAKE GENERAL RE...,2000-05-08,4500.0,DONE,BUILDING RESIDENTIAL - ADDITION,NaN,ACTIVITY-TYPE: A02 PURPOSE: TO CONSTRUCT A 22'...,1990-08-31,29700.0,DONE,ACREAGE TRACT,1991-08-16,30200.0,2473 TINNIN RD,37072,1973-07-26,SINGLE FAMILY,75800.0,171900.0,247700.0,18950.0,42975.0,61925.0,2021-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192178,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,188100G00100CO,2000 A OAK TRAIL DR 37135,TO CONSTRUCT A 2330 SQ FT UNIT WITH 136 SQ FT...,2017-12-14,NaN,DONE,BUILDING RESIDENTIAL - NEW,"ACCESSORY STRUCTURE, GARAGE",1/3/2018- MRP- RESTRICTIVE COVENANT FOR DETACH...,2017-12-14,NaN,DONE,CONDO,2017-06-23,48703.0,2000 A OAK TRAIL DR,37211.0,2017-11-15,RESIDENTIAL CONDO,57000.0,381500.0,438500.0,14250.0,128525.0,142775.0,2021-01-01
192179,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,188100G00200CO,2000 A OAK TRAIL DR 37135,TO CONSTRUCT A 2313 SQ FT UNIT WITH 37 SQ FT O...,2017-12-14,NaN,DONE,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,TO CONSTRUCT A 2313 SQ FT UNIT WITH 37 SQ FT O...,2017-12-14,NaN,DONE,CONDO,2020-08-14,320000.0,2000 B OAK TRAIL DR,37135.0,2017-11-15,RESIDENTIAL CONDO,57000.0,274200.0,331200.0,14250.0,87105.0,101355.0,2021-01-01
192180,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,188100G00200CO,2000 A OAK TRAIL DR 37135,TO CONSTRUCT A 2313 SQ FT UNIT WITH 37 SQ FT O...,2017-12-14,NaN,DONE,BUILDING RESIDENTIAL - NEW,"ACCESSORY STRUCTURE, GARAGE",1/3/2018- MRP- RESTRICTIVE COVENANT FOR DETACH...,2017-12-14,NaN,DONE,CONDO,2020-08-14,320000.0,2000 B OAK TRAIL DR,37135.0,2017-11-15,RESIDENTIAL CONDO,57000.0,274200.0,331200.0,14250.0,87105.0,101355.0,2021-01-01
192181,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,188100G00300CO,2000 A OA

In [18]:
# Modify date columns to date time types
Q2_v2[['dateissued', 'dateissued_imp']] = Q2_v2[['dateissued', 'dateissued_imp']].apply(pd.to_datetime)


# Want to create column to show duration from permit issued for demolition to permit issued for improvement
Q2_v2['Duration'] = (Q2_v2['dateissued_imp'] - Q2_v2['dateissued']).dt.days


In [19]:
#filter data based on 2 years or less (730 days) from demolition permit to improvement permit
# This is to ensure we capture only permit that have demolition that was done in order to improve property.
#Q2_2_years = Q2[Q2['Duration'] <= 730 ]
Q2_2_years_v2 = Q2_v2[(Q2_v2.Duration <= 730) & (Q2_v2.Duration >= 0) & (Q2_v2.permittype_imp =="BUILDING RESIDENTIAL - NEW")]

In [20]:
Q2_2_years_v2["location_zip"] = Q2_2_years_v2["location"].str[-5:]

<ipython-input-20-e15630cf4965>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q2_2_years_v2["location_zip"] = Q2_2_years_v2["location"].str[-5:]


In [21]:
# sort by location
Q2_2_years_v2.sort_values("location")

,permittype,permitsubtype,apn1,location,purpose,dateissued,value,status,permittype_imp,permitsubtype_imp,purpose_imp,dateissued_imp,value_imp,status_imp,featuretype,owndate,saleprice,propaddr,ownzip,propdate,ludesc,landappr,imprappr,totlappr,landassd,imprassd,totlassd,assessdate,Duration,location_zip
169313,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,150150B00100CO,**108 PAINTER DR *PER BONNIE @ PUBLIC WORKS,TO CONSTRUCT A 1737 SQ FT SINGLE FAMILY RESIDE...,2006-09-12,143214.66,EXPIRED,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,TO CONSTRUCT A 1737 SQ FT SINGLE FAMILY RESIDE...,2006-09-12,143214.66,EXPIRED,LOT,2017-09-19,255000.0,108 PAINTER DR,37013.0,2006-07-26,SINGLE FAMILY,50000.0,211400.0,261400.0,12500.0,52850.0,65350.0,2021-01-01,0.0,WORKS
113794,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,11000005000,0 HESSEY RD 37122,"FOR EVERY 30 FEET OF STREET FRONTAGE, OR FRACT...",2013-03-14,455089.13,DONE,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,"FOR EVERY 30 FEET OF STREET FRONTAGE, OR FRACT...",2013-03-14,455089.13,DONE,ACREAGE TRACT,2012-11-13,210000.0,0 HESSEY RD,37122.0,2013-03-14,VACANT RESIDENTIAL LAND,115300.0,0.0,115300.0,28825.0,0.0,28825.0,2021-01-01,0.0,37122
3864,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,03600006800,0 LITTLE MARROWBONE RD 37015,WORK ON THIS DEMOLITION PERMIT MUST COMMENCE W...,2018-06-13,3000.00,EXPIRED,BUILDING RESIDENTIAL - NEW,MOBILE HOME DWELLING,***HISTORY OF PROPERTY***\r\n1....ORIGINALLY 1...,2018-09-11,NaN,EXPIRED,ACREAGE TRACT,2007-04-05,0.0,5051 LITTLE MARROWBONE RD,37015,2007-04-05,MOBILE HOME,93700.0,7300.0,101000.0,23425.0,1825.0,25250.0,2021-01-01,90.0,37015
22963,ZONING BOARD APPEAL,SINGLE FAMILY RESIDENCE,06209007300,0 PENNINGTON BEND RD 37214,TO CONSTRUCT A 1861SF SINGLE FAMILY RESIDENCE ...,2008-03-19,NaN,EXPIRED,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,TO CONSTRUCT A 1861SF SINGLE FAMILY RESIDENCE ...,2008-05-02,199845.14,DONE,OTHER,2007-05-30,120000.0,2210 PENNINGTON BEND RD,37214,2008-02-08,SINGLE FAMILY,130000.0,285300.0,415300.0,52000.0,114120.0,166120.0,2021-01-01,44.0,37214
133550,BUILDING DEMOLITION PERMIT,DEMOLITION PERMIT - RESIDENTIAL,118060Y00100CO,0 UNKNOWN 00000,TO DEMOLISH EXISTING SINGLE FAMILY RESIDENCE. ...,2017-03-07,7500.00,EXPIRED,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,TO CONSTRUCT 841 A BATTLEFIELD DR. OF NEW HPR/...,2018-01-04,NaN,DONE,CONDO,2020-11-20,1049900.0,841 A BATTLEFIELD DR,37204.0,2020-11-04,RESIDENTIAL CONDO,224000.0,826900.0,1050900.0,56000.0,206725.0,262725.0,2021-01-01,303.0,00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54167,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,083034L00400CO,999 P RIVERSIDE DR 37206,MASTER PERMIT 2018040267\r\n\r\nTO CONSTRUCT 2...,2018-10-10,NaN,EXPIRED,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,MASTER PERMIT 2018040267\r\n\r\nTO CONSTRUCT 2...,2018-10-10,NaN,EXPIRED,CONDO,2013-09-25,NaN,999 P RIVERSIDE DR,37208.0,2020-08-26,VACANT RESIENTIAL LAND,40000.0,0.0,40000.0,10000.0,0.0,10000.0,2021-01-01,0.0,37206
148163,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,131031G00100CO,LONE OAK RD 37215,"FOR EVERY 30 FEET OF STREET FRONTAGE, OR FRACT...",2012-09-06,288356.08,DONE,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,"FOR EVERY 30 FEET OF STREET FRONTAGE, OR FRACT...",2012-09-06,288356.08,DONE,CONDO,2013-03-29,525000.0,4121 LONE OAK RD,37215,2012-08-30,RESIDENTIAL CONDO,272000.0,491500.0,763500.0,68000.0,122875.0,190875.0,2021-01-01,0.0,37215
148164,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,131031G00200CO,LONE OAK RD 37215,"FOR EVERY 30 FEET OF STREET FRONTAGE, OR FRACT...",2012-09-06,288356.08,DONE,BUILDING RESIDENTIAL - NEW,SINGLE FAMILY RESIDENCE,"FOR EVERY 30 FEET OF STREET FRONTAGE, OR FRACT...",2012-09-06,288356.08,DONE,CONDO,2013-04-29,525000.0,4123 LONE OAK RD,37215,2012-08-30,RESIDENTIAL CONDO,272000.0,517400.0,789400.0,68000.0,12

In [22]:
# Extract Year from Date/Time datatype
Q2_2_years_v2['year'] = Q2_2_years_v2['dateissued_imp'].dt.year

<ipython-input-22-b35c9d026cd9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q2_2_years_v2['year'] = Q2_2_years_v2['dateissued_imp'].dt.year


In [23]:
# Aggreagate by zipcode and year
Q2_2_years_agg_v2 = Q2_2_years_v2[['location_zip', 'year','permittype']].groupby(['location_zip', 'year']).agg(['count']).reset_index()

In [24]:
Q2_2_years_agg_v2

,location_zip,year,permittype
,,,count
0,10',2004,1
1,10-2,1987,1
2,20',2002,1
3,20',2005,1
4,241,2006,1
...,...,...,...
1223,Y AVE,2016,4
1224,Y AVE,2017,1
1225,Y COV,2001,1


In [25]:
# slice out non-zipcode data
Q2_2_years_agg_v2 = Q2_2_years_agg_v2[Q2_2_years_agg_v2['location_zip'].str.contains(r'^37\d{3}$')]

In [30]:
#Filter zipcodes for year 2015
Q2_agg_2015 = Q2_2_years_agg_v2[Q2_2_years_agg_v2['year']==2015].sort_values(('permittype', 'count'),ascending=False)

In [31]:
#Filter zipcodes for year 2017
Q2_agg_2017 = Q2_2_years_agg_v2[Q2_2_years_agg_v2['year']==2017].sort_values(('permittype', 'count'),ascending=False)

In [32]:
#Filter zipcodes for year 2020
Q2_agg_2020 = Q2_2_years_agg_v2[Q2_2_years_agg_v2['year']==2020].sort_values(('permittype', 'count'),ascending=False)

In [33]:
#Filter zipcodes for year 2021
Q2_agg_2021 = Q2_2_years_agg_v2[Q2_2_years_agg_v2['year']==2021].sort_values(('permittype', 'count'),ascending=False)

In [37]:
Q2_agg_2017_2020 = pd.merge(Q2_agg_2017, Q2_agg_2020, left_on='location_zip', right_on='location_zip')

In [61]:
Q2_agg_2017_2020.columns = Q2_agg_2017_2020.columns.droplevel(1)

In [64]:
Q2_agg_2017_2020

,location_zip,year_x,permittype_x,year_y,permittype_y
0,37209,2017,704,2020,535
1,37013,2017,465,2020,530
2,37208,2017,314,2020,180
3,37207,2017,290,2020,284
4,37206,2017,252,2020,184
5,37135,2017,223,2020,187
6,37203,2017,216,2020,76
7,37216,2017,180,2020,85
8,37076,2017,177,2020,176
9,37215,2017,168,2020,108


In [67]:
Q2_agg_2017_2020 = Q2_agg_2017_2020.drop('year_y',axis=1)

In [70]:
Q2_agg_2017_2020[['permittype_x','permittype_y']] = Q2_agg_2017_2020[['permittype_x','permittype_y']].astype('int32')

In [81]:
Q2_agg_2017_2020['percentage_change'] = ((Q2_agg_2017_2020['permittype_y'] - Q2_agg_2017_2020['permittype_x'])/ Q2_agg_2017_2020['permittype_x'])*100

In [82]:
Q2_agg_2017_2020.sort_values('percentage_change', ascending= False)

,location_zip,year_x,permittype_x,permittype_y,percentage_change
25,37080,2017,9,53,488.888889
20,37027,2017,20,86,330.000000
23,37217,2017,16,38,137.500000
27,37015,2017,1,2,100.000000
13,37211,2017,92,150,63.043478
17,37138,2017,66,97,46.969697
21,37214,2017,18,23,27.777778
1,37013,2017,465,530,13.978495
18,37218,2017,49,54,10.204082
24,37220,2017,11,11,0.000000
